In [7]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pickle

In [8]:
def write_pickle(object, file_name):
    with open(file_name, "wb") as handle:
        pickle.dump(object, handle, protocol=pickle.HIGHEST_PROTOCOL)

def read_pickle(path):
    with open(path, 'rb') as handle:
        return pickle.load(handle)

# Load raw data

In [9]:
df_web = pd.read_csv('text_website.csv', header=None).rename(columns={0: "label", 1: "text"})
df_tiktok = pd.read_csv('text_tiktok.csv', header=None).rename(columns={0: "label", 1: "text"})

In [140]:
df_web.head()

,label,text
0,1,Với quyết định trên Hà Nội là địa phương đầu t...
1,1,Sáng 254 Ủy ban Thường vụ Quốc hội cho ý kiến ...
2,1,Theo thông cáo chiều 214 tại kỳ họp thứ 14 192...
3,1,Thường trực Ban Bí thư Võ Văn Thưởng vừa ký kế...
4,1,Theo thông cáo của Văn phòng Trung ương Đảng c...


In [141]:
df_tiktok.head()

,label,text
0,1,12 luật cấm kỳ lạ ở Triều Tiên khiến cả thế g...
1,1,Việt Nam ơi giống vùng kín Ngàn Năm lưu dành ...
2,1,dưới cờ đỏ sao vàng thiêng liêng của tổ quốc ...
3,1,quốc gia nào trong khu vực Đông Nam Á được th...
4,1,không là người yêu cũ của ai cả Bởi vì nếu nh...


# Preprocessing

In [12]:
import re
from underthesea import word_tokenize
# vncorenlp
# from pyvi import ViTokenizer

def is_number(text):
    try:
        float(text)
        return True
    except ValueError:
        return False

def preprocess_text(text):    
    text = re.sub(r'<[^>]*>', '', text)
    text = text.lower()
    text = word_tokenize(text, format="text")
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_\[\]]',' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = ' '.join(map(lambda x: '<number>' if is_number(x) else x, text.split()))
    return text

## processing data web & tiktok

In [11]:
df_web['preprocessed_text'] = df_web['text'].apply(preprocess_text)
df_tiktok['preprocessed_text'] = df_tiktok['text'].apply(preprocess_text)

KeyboardInterrupt: 

In [ ]:
df_web[['label', 'preprocessed_text']].to_csv('preprocessed_web.csv', index=False)
df_tiktok[['label', 'preprocessed_text']].to_csv('preprocessed_tiktok.csv', index=False)

# Load preprocessed data

In [13]:
df_web = pd.read_csv('preprocessed_web.csv')
df_tiktok = pd.read_csv('preprocessed_tiktok.csv')

In [6]:
df_web.head()

,label,preprocessed_text
0,1,với quyết_định trên hà_nội là địa_phương đầu_t...
1,1,sáng 254_ủy_ban thường_vụ quốc_hội cho ý_kiến ...
2,1,theo thông_cáo chiều <number> tại kỳ họp thứ <...
3,1,thường trực_ban bí_thư võ văn thưởng vừa ký_kế...
4,1,theo thông_cáo của văn_phòng trung_ương đảng c...


In [7]:
df_tiktok.head()

,label,preprocessed_text
0,1,<number> luật cấm kỳ_lạ ở triều tiên khiến cả ...
1,1,việt nam ơi giống vùng kín ngàn năm lưu dành s...
2,1,dưới cờ đỏ sao_vàng thiêng_liêng của tổ_quốc t...
3,1,quốc_gia nào trong khu_vực đông nam á được thể...
4,1,không là người_yêu cũ của ai cả bởi_vì nếu_như...


### Split data train & test

In [14]:
from sklearn.model_selection import train_test_split

df_web_train, df_web_test = train_test_split(df_web, test_size=0.2, random_state=2022, stratify=df_web['label'])
df_tiktok_train, df_tiktok_test = train_test_split(df_tiktok, test_size=0.2, random_state=2022, stratify=df_tiktok['label'])

In [15]:
df_train = pd.concat([df_web_train, df_tiktok_train], axis=0)
df_test = pd.concat([df_web_test, df_tiktok_test], axis=0)

### Prepare data for fasttext

In [16]:
df_train['fasttext_label'] = '__label__' + df_train['label'].astype(str) 

In [34]:
df_train[['fasttext_label', 'preprocessed_text']].head()

,fasttext_label,preprocessed_text
3857,__label__0,do lượng đăng_ký dự vm quy_nhơn tăng mạnh nhữn...
6572,__label__0,cầu_thủ saudi arabia áo trắng trong vòng_vây c...
11904,__label__0,mức lợi_nhuận hợp_nhất nửa đầu năm mà ngân_hàn...
5113,__label__1,với <number> lượt ý_kiến thảo_luận tại tổ và h...
12600,__label__0,thủ_tướng phạm minh chính tiếp đoàn doanh_nghi...


In [31]:
df_train[['fasttext_label', 'preprocessed_text']].to_csv('train.txt', header=None, index=None, sep='#')

# Featurize (tf-idf)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
# https://scikit-learn.org/0.24/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html?highlight=tfidf#sklearn.feature_extraction.text.TfidfVectorizer

In [18]:
corpus = pd.concat([df_web['preprocessed_text'], df_tiktok['preprocessed_text']])

In [34]:
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

TfidfVectorizer()

In [141]:
# write_pickle(vectorizer, 'model_text/tfidfVectorizer')

# Model

## Load TF-IDF

In [19]:
vectorizer = read_pickle('model_text/tfidfVectorizer')

In [20]:
x_train = vectorizer.transform(df_train['preprocessed_text'])
y_train = df_train['label']

x_test = vectorizer.transform(df_test['preprocessed_text'])
y_test = df_test['label']

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(random_state=2022)
clf.fit(x_train, y_train)

In [147]:
write_pickle(clf, 'model_text/rf_clf')

## Logistic Regression

In [151]:
from sklearn.linear_model import LogisticRegression

In [152]:
clf = LogisticRegression(random_state=0)
clf.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [153]:
write_pickle(clf, 'model_text/lr_clf')

## Naive Bayes

In [66]:
from sklearn.naive_bayes import MultinomialNB

In [67]:
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB()

In [68]:
write_pickle(clf, 'model_text/nb_clf')

## SVM

In [69]:
from sklearn.svm import SVC

In [70]:
clf = SVC(gamma='scale')
clf.fit(x_train, y_train)

SVC()

In [72]:
write_pickle(clf, 'model_text/svm_clf')

## Fasttext

In [35]:
import fasttext

In [36]:
clf = fasttext.train_supervised(
                                input='train.txt',
                                dim=100,
                                epoch=5,
                                lr=0.1,
                                wordNgrams=1,
                                label='__label__',
                                minCount=5)

Read 6M words
Number of words:  23884
Number of labels: 2
Progress: 100.0% words/sec/thread: 4243698 lr:  0.000000 avg.loss:  0.176556 ETA:   0h 0m 0s


In [37]:
clf.quantize(input='train.txt', retrain=True)
clf.save_model("model_text/fasttext.ftz")

## Evaluation

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from utils import calculate_accuracy, calculate_f1_score

### machine learning model

#### Random forest model

In [14]:
rf_clf = read_pickle('model_text/rf_clf') 
y_pred_rf = rf_clf.predict(x_test)

In [15]:
calculate_accuracy(y_test, y_pred_rf)

0.962598833441348

In [16]:
calculate_f1_score(y_test, y_pred_rf)

0.9597771775827144

#### Logistic regression

In [17]:
lr_clf = read_pickle('model_text/lr_clf')
y_pred_lr = lr_clf.predict(x_test)

In [18]:
calculate_accuracy(y_test, y_pred_lr)

0.9807453013609851

In [19]:
calculate_f1_score(y_test, y_pred_lr)

0.978122866894198

#### Naive bayes

In [20]:
nb_clf = read_pickle('model_text/nb_clf')
y_pred_nb = nb_clf.predict(x_test)

In [21]:
calculate_accuracy(y_test, y_pred_nb)

0.9519053791315619

In [22]:
calculate_f1_score(y_test, y_pred_nb)

0.9499601461308536

#### SVM

In [23]:
svm_clf = read_pickle('model_text/svm_clf')
y_pred_svm = svm_clf.predict(x_test)

In [24]:
calculate_accuracy(y_test, y_pred_svm)

0.9862540505508749

In [25]:
calculate_f1_score(y_test, y_pred_svm)

0.9840972458347501

### fasttext

In [38]:
clf = fasttext.load_model("model_text/fasttext.ftz")

In [39]:
def predict_fasttext(text):
    prediction = clf.predict(text)
    return int(prediction[0][0][-1])

In [40]:
y_pred = df_test['preprocessed_text'].apply(predict_fasttext)

In [41]:
calculate_accuracy(df_test['prediction'], df_test['label'])

KeyError: 'prediction'

In [122]:
f1_score(df_test['prediction'], df_test['label'])

0.9475806451612904

In [3]:
# tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
# confusion_matrix(df_test['prediction'], df_test['label']).ravel()

# Prediction

### Random Forest, Logistic Regression, SVM, NB

In [22]:
vectorizer = read_pickle('model_text/tfidfVectorizer') # load vectorizer
# clf = read_pickle('model_text/rf_clf') 
# clf = read_pickle('model_text/lr_clf')
# clf = read_pickle('model_text/nb_clf')
clf = read_pickle('model_text/svm_clf')

In [31]:
# raw_text = 'Không bao giờ chia sẻ về thu nhập của mình Đức Bảo sẽ là một trong 3 người dẫn các chương trình'
raw_text = 'Thủ tướng Nguyễn Bình Long vừa ký quyết định số 547'
preprocessed_text = preprocess_text(raw_text)
vector_tf_idf = vectorizer.transform([preprocessed_text])

In [32]:
clf.predict(vector_tf_idf)

array([1])

### Fasttext

In [6]:
clf = fasttext.load_model("model_text/fasttext.ftz")

In [6]:
# raw_text = 'Không bao giờ chia sẻ về thu nhập của mình Đức Bảo sẽ là một trong 3 người dẫn các chương trình'
raw_text = 'Phó Thủ tướng Lê Minh khái vừa ký quyết định số 547 do Bộ Tài chính thức cấp không thu tiền 139608 tấn gạo từ nguồn dự trữ quốc gia cho 2 tỉnh Tuyên Quang Quảng Trị để hỗ trợ nhân dân trong thời gian rất hạn đầu năm 2022 theo quyết định tỉnh Tuyên quanh được sức cấp 3108 tấn gạo tỉnh Quảng Trị 108528 tấn gạo Phó Thủ tướng yêu cầu Bộ Tài chính Bộ Lao động Thương binh và xã hội xử lý cụ thể theo quy định chịu trách nhiệm về thông tin và số liệu báo cáo Ủy ban Nhân dân hai tỉnh Tuyên quanh Quảng Trị chịu trách nhiệm về tính chính xác của số liệu báo cáo và thực hiện hỗ trợ kịp thời đúng đối tượng định mức theo quy định'
preprocessed_text = preprocess_text(raw_text)
clf.predict(preprocessed_text)

NameError: name 'clf' is not defined